In [1]:
from rdflib import URIRef, BNode, Literal, Namespace
from rdflib.namespace import RDF, SDO, RDFS
from rdflib import Graph

#14	Nintendo shop - https://store.nintendo.nl/nl
#Zijian Dong -i6226172	Anqi Fan -i6272224	Qi Cui -i6288444	Tianyu Wei -i6231911	Zhefan Cheng -i6267501


In [2]:
# Make a graph
g = Graph()
NS = Namespace('https://store.nintendo.nl/nl/')
nintendo = Literal("Nintendo")
#--------------Categories------------------------------------

console = NS['consoles']
games = NS['games']
merchandise = NS['merchandise'] #周边merchandise
franchise = NS['franchises'] #独家ip 和其它的会重复Exclusive IP and others will be duplicated
new_product = NS['new']

#--------Sub-Categories--------------------------------------
#--------Sub Categories for consoles-------------------------
switch = NS['consoles/nintendo-switch-consoles'] #是console的product
switch_bundle = NS['consoles/nintendo-switch-bundles'] #是console的product
accessory = NS['consoles/accessories'] #是switch的配件
#-------Sub Categories for merchandise-----------------
apparel = NS['merchandise/apparel']
home_and_gifts = NS['merchandise/home-and-gifts']


In [3]:
#--------Instance--------------------------------------

#Define types for Mainpage
g.add((console, RDF.type, nintendo)) # add的三个参数："_SubjectType", "_PredicateType", "_ObjectType"
g.add((games, RDF.type, nintendo))
g.add((merchandise, RDF.type, nintendo))
g.add((franchise, RDF.type, nintendo))
g.add((new_product, RDF.type, nintendo))

#Define subclasses of main categories
g.add((switch, RDFS.subClassOf, console))
g.add((switch_bundle, RDFS.subClassOf, console))
g.add((accessory, SDO.isAccessoryOrSparePartFor, console))

g.add((apparel, RDFS.subClassOf, merchandise))
g.add((home_and_gifts, RDFS.subClassOf, merchandise))


<Graph identifier=N41552e07f08a4ac78b61b5b520c98cfc (<class 'rdflib.graph.Graph'>)>

In [4]:
import pandas as pd

df = pd.read_excel("data.xlsx")

for i,row in df.iterrows():
    # Check if URI is empty, skip the empty ones
    if not pd.isna(row["Item_URI"]) :
        item = NS[row["Item_URL"].replace("https://store.nintendo.nl/nl/","").replace(" ","")]
        for category in row['type'].split(","):
            g.add((item, SDO.product, locals()[category]))
        g.add((item, SDO.name, Literal(row["Value_1"])))
        g.add((item, SDO.description, Literal(row["Value_2"])))
        g.add((item, SDO.sku, Literal(row["Value_3"])))
        g.add((item, SDO.price, Literal(row["Value_4"])))
        g.add((item, SDO.brand, Literal("Nintendo"))) 
    #    graph.add((product, schema[row["link"]],eval(row["type"])))


C:\Users\AnkieF\AppData\Local\Temp\ipykernel_30432\3898633231.py:10: UserWarning: Code: product is not defined in namespace SDO
  g.add((item, SDO.product, locals()[category]))


In [5]:
g.bind('NS', NS)

print(g.serialize('Nintendo.ttl',format='turtle'))

[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].


## SPARQL:

In [6]:
from rdflib import Graph

graph = Graph()
graph.parse("Nintendo.ttl", format="turtle")

<Graph identifier=N60e3edab5dc14eaf91956292cb8425ee (<class 'rdflib.graph.Graph'>)>

In [8]:
'''
AN EXAMPLE
'''
item_info_query = """
SELECT ?price WHERE {
  ?s schema:name 'Splatoon 3'.
  ?s schema:price ?price.
}
"""

results = graph.query(item_info_query)
for row in results:
    print(row.price)

59.99


In [ ]:
# Check the full graph: 查看所有语句 很长 没事别run 需要检查的时候再跑
query = """
SELECT * WHERE {
  ?s ?p ?o
}
"""
results = graph.query(query)
for row in results:
    print(row)